In [6]:
import torch
import copy
import numpy as np
from torchvision import transforms
from torchvision.transforms.functional import to_pil_image
from torchinfo import summary
import cv2
import matplotlib.pyplot as plt
from deepmag import dataset, viz, model, train

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [2]:
torch.cuda.get_device_name()

'Orin'

In [27]:
model = torch.load('data/models/20191202-b10-r0.1-lr0.0001-00.pt')

/usr/local/lib/python3.8/dist-packages/torch/serialization.py:888: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.8/dist-packages/torch/serialization.py:888: SourceChangeWarning: source code of class 'torch.nn.modules.padding.ReflectionPad2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.8/dist-packages/torch/serialization.py:888: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute

In [28]:
m = copy.deepcopy(model)
m.eval()

MagNet(
  (encoder): Encoder(
    (conv): Sequential(
      (0): ReflectionPad2d((3, 3, 3, 3))
      (1): Conv2d(3, 16, kernel_size=(7, 7), stride=(1, 1))
      (2): ReLU()
      (3): ReflectionPad2d((1, 1, 1, 1))
      (4): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
      (5): ReLU()
    )
    (res): Sequential(
      (0): ResidualBlock(
        (block): Sequential(
          (0): ReflectionPad2d((1, 1, 1, 1))
          (1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
          (2): ReLU()
          (3): ReflectionPad2d((1, 1, 1, 1))
          (4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
        )
      )
      (1): ResidualBlock(
        (block): Sequential(
          (0): ReflectionPad2d((1, 1, 1, 1))
          (1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
          (2): ReLU()
          (3): ReflectionPad2d((1, 1, 1, 1))
          (4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
        )
      )
      (2): ResidualBlock(
        (block): Seque

In [29]:
m = torch.nn.Sequential(torch.quantization.QuantStub(), 
                  m, 
                  torch.quantization.DeQuantStub())

In [30]:
backend = "fbgemm"

m.qconfig = torch.quantization.get_default_qconfig(backend)
torch.quantization.prepare(m, inplace=True)

/usr/local/lib/python3.8/dist-packages/torch/ao/quantization/observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


Sequential(
  (0): QuantStub(
    (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
  )
  (1): MagNet(
    (encoder): Encoder(
      (conv): Sequential(
        (0): ReflectionPad2d((3, 3, 3, 3))
        (1): Conv2d(
          3, 16, kernel_size=(7, 7), stride=(1, 1)
          (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
        )
        (2): ReLU()
        (3): ReflectionPad2d((1, 1, 1, 1))
        (4): Conv2d(
          16, 32, kernel_size=(3, 3), stride=(2, 2)
          (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
        )
        (5): ReLU()
      )
      (res): Sequential(
        (0): ResidualBlock(
          (block): Sequential(
            (0): ReflectionPad2d((1, 1, 1, 1))
            (1): Conv2d(
              32, 32, kernel_size=(3, 3), stride=(1, 1)
              (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
            )
            (2): ReLU()
            (3): Reflectio

In [34]:
with torch.inference_mode():
    for _ in range(10):
        example_input_0 = torch.rand((1, 3, 120, 160), dtype=torch.float32)
        example_input_1 = torch.rand((1, 3, 120, 160), dtype=torch.float32)
        example_input_2 = torch.full((1, 1), 5, dtype=torch.float32)
        m(example_input_0, example_input_1, example_input_2)

TypeError: forward() takes 2 positional arguments but 4 were given